# Cleaning & Merging NBA Player Stats (1998-2025) & Salary Data (2000-2025)

### Libraries

In [965]:
import pandas as pd
import unicodedata

## Salary Data (2000-2025)

#### 2000-2020
Much thanks to **erikgregorywebb**, [GitHub link](https://github.com/erikgregorywebb/datasets/blob/master/nba-salaries.csv)

In [968]:
salary_2000_2020 = pd.read_csv('salaries_2000-2020.csv') 
salary_2000_2020

,Player,Salary,Season
0,Shaquille O'Neal,17142000,2000
1,Kevin Garnett,16806000,2000
2,Alonzo Mourning,15004000,2000
3,Juwan Howard,15000000,2000
4,Scottie Pippen,14795000,2000
...,...,...,...
9451,Jimmer Fredette,208509,2020
9452,Jontay Porter,197933,2020
9453,Anthony Tolliver,183115,2020
9454,Tyler Johnson,183115,2020


#### 2021-2024
scrapped from [HoopsHype](https://hoopshype.com/salaries/players/2020-2021/)

In [970]:
seasons = {
    '2020-2021': '2021',
    '2021-2022': '2022',
    '2022-2023': '2023',
    '2023-2024': '2024',
}

In [971]:
all_salary_data = []

for url_season, season_label in seasons.items():
    url = f'https://hoopshype.com/salaries/players/{url_season}/'
    try:
        tables = pd.read_html(url)
        salary_df = tables[0]
        salary_df.columns = ['Rank', 'Player', 'Team', 'Salary']
        salary_df = salary_df[['Player', 'Salary']].copy()
        salary_df['Season'] = season_label
        salary_df['Salary'] = salary_df['Salary'].replace(r'[\$,]', '', regex=True).astype(float).astype(int)
        all_salary_data.append(salary_df)
    except Exception as e:
        print(f"Error processing {url}: {e}")

Merging

In [973]:
merged_salary = pd.concat(all_salary_data, ignore_index=True)

# Preview the result
print(merged_salary)

                  Player    Salary Season
0          Stephen Curry  52411485   2021
1             Chris Paul  50403633   2021
2      Russell Westbrook  50403633   2021
3           James Harden  50277018   2021
4              John Wall  50277018   2021
...                  ...       ...    ...
2412      Jalen Crutcher     66254   2024
2413  Izaiah Brockington     66254   2024
2414      Taevion Kinsey     66254   2024
2415       Edmond Sumner     41661   2024
2416        Kaiser Gates     36440   2024

[2417 rows x 3 columns]


In [974]:
merged_salary.to_csv('nba_salaries_2021-24', index=False)

salary_2021_2024 = pd.read_csv('salaries_2021-2024.csv')

#### 2025
Already scrapped from [HoopsHype](https://hoopshype.com/salaries/players/)

In [976]:
salary_2024_25 = pd.read_csv('salaries_2025.csv')

Removing the dollar sign and commas

In [978]:
salary_2024_25['Salary'] = salary_2024_25['Salary'].replace(r'[\$,]', '', regex=True).astype(int)

print(salary_2024_25)

            Player    Salary  Season
0    Stephen Curry  55761216    2025
1      Joel Embiid  51415938    2025
2     Nikola Jokic  51415938    2025
3     Kevin Durant  51179021    2025
4     Bradley Beal  50203930    2025
..             ...       ...     ...
558  Isaiah Mobley     11997    2025
559    Braxton Key     11997    2025
560     JD Davison     11997    2025
561   Kylor Kelley     73153    2025
562    Jaylen Sims     73153    2025

[563 rows x 3 columns]


#### Merging 2000-2025 Datasets

Double check for dollar signs

In [981]:
for df in [salary_2000_2020, salary_2021_2024, salary_2024_25]:
    df['Salary'] = df['Salary'].replace(r'[\$,]', '', regex=True).astype(int)

Merging

In [983]:
merged_salary = pd.concat([salary_2000_2020, salary_2021_2024, salary_2024_25], ignore_index=True)

Extra check for duplicates

In [985]:
merged_salary.drop_duplicates(subset=['Player', 'Season'], keep='last', inplace=True)

I will integrate season column as year, e.g. 2024-25 season as year '2025'

In [987]:
merged_salary.rename(columns={'Season': 'Year'}, inplace=True)

In [988]:
print(merged_salary)
print(f"Total rows after merge: {len(merged_salary)}")

                 Player    Salary  Year
0      Shaquille O'Neal  17142000  2000
1         Kevin Garnett  16806000  2000
2       Alonzo Mourning  15004000  2000
3          Juwan Howard  15000000  2000
4        Scottie Pippen  14795000  2000
...                 ...       ...   ...
12431     Isaiah Mobley     11997  2025
12432       Braxton Key     11997  2025
12433        JD Davison     11997  2025
12434      Kylor Kelley     73153  2025
12435       Jaylen Sims     73153  2025

[12386 rows x 3 columns]
Total rows after merge: 12386


In [989]:
merged_salary.to_csv('nba_salaries_2000-2025', index=False)

Done!

## Stats Data (1998-2025)

#### 1998-2022
Much thanks to **Patrick_Batman**, [Kaggle Link](https://www.kaggle.com/datasets/raunakpandey030/nba-player-stats?select=NBA_Player_Stats.csv)

In [993]:
stats_2 = pd.read_csv("NBA Player Stats_1998-2022.csv") 

#### 2023-2025
Data downloaded from [Basketball Reference](https://www.basketball-reference.com/leagues/NBA_2025_per_game.html)

In [995]:
stats_2023 = pd.read_csv("stats_2023.csv")
stats_2024 = pd.read_csv("stats_2024.csv")   
stats_2025 = pd.read_csv("stats_2025.csv")

### Cleaning before merging

In [997]:
print(stats_2.columns)
print(stats_2023.columns)
print(stats_2024.columns)
print(stats_2025.columns)

Index(['Rk', 'Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%',
       '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%',
       'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Year'],
      dtype='object')
Index(['Rk', 'Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%',
       '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%',
       'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS'],
      dtype='object')
Index(['Rk', 'Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%',
       '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%',
       'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS'],
      dtype='object')
Index(['Rk', 'Player', 'Age', 'Team', 'Pos', 'G', 'GS', 'MP', 'FG', 'FGA',
       'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA',
       'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS'],
      dtype='object')


1) need to add 'Year' for 2023-2025
2) stats order for 2025 doesn't match other datasets
3) 'Team' for 2025, 'Tm' for 1998-2024

Adding Years 

In [1000]:
stats_2023['Year'] = 2023
stats_2024['Year'] = 2024
stats_2025['Year'] = 2025

Change 2025 column to 'Tm' to merge

In [1002]:
stats_2025.rename(columns={'Team': 'Tm'}, inplace=True)

Match column orders

In [1004]:
cols = stats_2.columns.tolist()

In [1005]:
stats_2023 = stats_2023[cols]
stats_2024 = stats_2024[cols]
stats_2025 = stats_2025[cols]

Final check

In [1007]:
print(stats_2.columns)
print(stats_2023.columns)
print(stats_2024.columns)
print(stats_2025.columns)

Index(['Rk', 'Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%',
       '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%',
       'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Year'],
      dtype='object')
Index(['Rk', 'Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%',
       '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%',
       'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Year'],
      dtype='object')
Index(['Rk', 'Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%',
       '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%',
       'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Year'],
      dtype='object')
Index(['Rk', 'Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%',
       '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%',
       'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Year'],
  

for 1998-2022, 'Year' is shown as season e.g. 1998-2022 dataset: Year = "1998-1999". <br>
so, only leave the latter half (remove string '1998-' from '1998-1999')

In [1009]:
def extract_latter_year(x):
    try:
        if isinstance(x, str) and '-' in x:
            return int(x.split('-')[-1])
        return int(x)
    except:
        return pd.NA

In [1010]:
stats_2['Year'] = stats_2['Year'].apply(extract_latter_year)

print(stats_2['Year'])

0        1998
1        1998
2        1998
3        1998
4        1998
         ... 
14568    2022
14569    2022
14570    2022
14571    2022
14572    2022
Name: Year, Length: 14573, dtype: int64


Check for NaNs

In [1012]:
print("NaNs in Year column:", stats_2['Year'].isna().sum())

NaNs in Year column: 0


Great!

#### Data types

In [1015]:
print(stats_2.dtypes)
print(stats_2023.dtypes)
print(stats_2024.dtypes)
print(stats_2025.dtypes)

Rk          int64
Player     object
Pos        object
Age         int64
Tm         object
G           int64
GS          int64
MP        float64
FG        float64
FGA       float64
FG%       float64
3P        float64
3PA       float64
3P%       float64
2P        float64
2PA       float64
2P%       float64
eFG%      float64
FT        float64
FTA       float64
FT%       float64
ORB       float64
DRB       float64
TRB       float64
AST       float64
STL       float64
BLK       float64
TOV       float64
PF        float64
PTS       float64
Year        int64
dtype: object
Rk          int64
Player     object
Pos        object
Age        object
Tm         object
G           int64
GS        float64
MP        float64
FG        float64
FGA       float64
FG%       float64
3P        float64
3PA       float64
3P%       float64
2P        float64
2PA       float64
2P%       float64
eFG%      float64
FT        float64
FTA       float64
FT%       float64
ORB       float64
DRB       float64
TRB       floa

'Age', 'GS', 'Year' columns should be 'Int', since it should be numeric value for further analysis

In [1017]:
datasets = [stats_2, stats_2023, stats_2024, stats_2025]

In [1018]:
for df in datasets:
    df['Age'] = pd.to_numeric(df['Age'], errors='coerce').astype('Int64')
    df['GS'] = pd.to_numeric(df['GS'], errors='coerce')
    if df['GS'].isna().sum() == 0:
        df['GS'] = df['GS'].astype('int64')
    df['Year'] = pd.to_numeric(df['Year'], errors='coerce').astype('int64')

#### Normalizing names (changing accented alphabets to normal alphabets)
e.g. Nikola Jokić to Nikola Jokic, Dāvis Bertāns to Davis Bertans <br>
necessary to do this, to search names conveniently

In [1020]:
def normalize_name(name):
    if isinstance(name, str):
        return unicodedata.normalize('NFKD', name).encode('ASCII', 'ignore').decode('utf-8')
    return name

In [1021]:
for df in [stats_2, stats_2023, stats_2024, stats_2025]:
    df['Player'] = df['Player'].apply(normalize_name)

Player names should be in ASCII values, too

In [1023]:
for df, label in zip([stats_2, stats_2023, stats_2024, stats_2025], 
                     ['1998-2022', '2023', '2024', '2025']):
    bad_names = df[df['Player'].str.contains(r'[^\x00-\x7F]', na=False)]
    if not bad_names.empty:
        print(f"Non-ASCII names found in {label}:")
        print(bad_names['Player'].drop_duplicates())

Check if it worked

In [1025]:
def search_player(name, df):
    result = df[df['Player'].str.contains(name, case=False, na=False)]
    if result.empty:
        print(f"No results found for '{name}'")
    else:
        print(result.to_string(index=False))

search_player("Jokic", stats_2025)
search_player("Nikola Jokic", stats_2025)

 Rk       Player Pos  Age  Tm  G  GS   MP   FG  FGA   FG%  3P  3PA   3P%  2P  2PA   2P%  eFG%  FT  FTA  FT%  ORB  DRB  TRB  AST  STL  BLK  TOV  PF  PTS  Year
  3 Nikola Jokic   C   29 DEN 70  70 36.7 11.2 19.5 0.576 2.0  4.7 0.417 9.3 14.8 0.627 0.627 5.2  6.4  0.8  2.9  9.9 12.7 10.2  1.8  0.6  3.3 2.3 29.6  2025
 Rk       Player Pos  Age  Tm  G  GS   MP   FG  FGA   FG%  3P  3PA   3P%  2P  2PA   2P%  eFG%  FT  FTA  FT%  ORB  DRB  TRB  AST  STL  BLK  TOV  PF  PTS  Year
  3 Nikola Jokic   C   29 DEN 70  70 36.7 11.2 19.5 0.576 2.0  4.7 0.417 9.3 14.8 0.627 0.627 5.2  6.4  0.8  2.9  9.9 12.7 10.2  1.8  0.6  3.3 2.3 29.6  2025


Perfect!

### Merge it, finally

In [1028]:
merged_stats = pd.concat([stats_2, stats_2023, stats_2024, stats_2025], ignore_index=True)

### Additional cleaning

In [1030]:
merged_stats.drop_duplicates(inplace=True)

Indexes were messed up after merging, so:

In [1032]:
merged_stats.reset_index(drop=True, inplace=True)

Some name entries had '*' or unnecessary whitespaces, so:

In [1034]:
merged_stats['Player'] = merged_stats['Player'].str.replace('*', '', regex=False).str.strip()

Made a mistake to put team as 'Tm', make it to 'Team' for future convenience and avoiding confusion

In [1036]:
merged_stats.rename(columns={'Tm': 'Team'}, inplace=True)

#### Additional Cleaning: Teams
There are players that played for multiple teams in the same season (e.g. Zach Lavine in 2024-25, traded from CHI to SAC) <br>
I figured out that datasets included multiple entries for these players (e.g. Zach Lavine has three, total stats for both, stats at CHI, and stats in SAC) <br>
So I needed to clean those so that it doesn't bias the dataset when I merge with salary dataset.

First, define the team codes manually: there were four: 'TOT', '2TM', '3TM', '4TM'. <br>

In [1039]:
total_team_codes = ['TOT', '2TM', '3TM', '4TM']

Step 1: Separate total rows and partial team rows

In [1041]:
total_rows = merged_stats[merged_stats['Team'].isin(total_team_codes)].copy()
partial_rows = merged_stats[~merged_stats['Team'].isin(total_team_codes)].copy()

Step 2: For each (Player, Year) in total_rows, find last team in partial_rows. 'last team' meaning the team that player played in the end of the season (e.g. SAC for Zach Lavine)

In [1043]:
def get_latest_team(player, year):
    subset = partial_rows[(partial_rows['Player'] == player) & (partial_rows['Year'] == year)]
    if not subset.empty:
        return subset.iloc[-1]['Team']
    else:
        return None

Step 3: apply function to total_rows to get latest teams & replace 'Team' column in total_rows

In [1045]:
total_rows['LatestTeam'] = total_rows.apply(lambda row: get_latest_team(row['Player'], row['Year']), axis=1)

total_rows['Team'] = total_rows.apply(lambda row: row['LatestTeam'] if pd.notna(row['LatestTeam']) else row['Team'], axis=1)

Drop helper column

In [1047]:
total_rows.drop(columns=['LatestTeam'], inplace=True)

Step 4: Combine total_rows with partial_rows filtered to exclude partial rows of players who have totals
i.e. remove partial rows for player-year if total row exists

In [1049]:
player_year_with_total = set(total_rows[['Player', 'Year']].itertuples(index=False, name=None))
partial_rows_filtered = partial_rows[~partial_rows.apply(lambda row: (row['Player'], row['Year']) in player_year_with_total, axis=1)]

Final: concat these two rows

In [1051]:
cleaned_stats = pd.concat([total_rows, partial_rows_filtered], ignore_index=True)

Optional: sort by Player, Year, Team

In [1053]:
cleaned_stats.sort_values(by=['Player', 'Year', 'Team'], inplace=True)

print(f"Original stats rows: {len(merged_stats)}")
print(f"Cleaned stats rows: {len(cleaned_stats)}")

Original stats rows: 16722
Cleaned stats rows: 13388


#### Additional checking for teams

In [1055]:
total_team_codes = ['TOT', '2TM', '3TM', '4TM']
total_rows = cleaned_stats[cleaned_stats['Team'].isin(total_team_codes)]
partial_rows = cleaned_stats[~cleaned_stats['Team'].isin(total_team_codes)]

In [1056]:
total_player_years = set(total_rows[['Player', 'Year']].itertuples(index=False, name=None))
partial_player_years = set(partial_rows[['Player', 'Year']].itertuples(index=False, name=None))

In [1057]:
overlap = total_player_years.intersection(partial_player_years)
print(f"Number of player-year pairs with both total and partial rows: {len(overlap)}")
if len(overlap) > 0:
    print("Examples of overlaps:", list(overlap)[:5])

Number of player-year pairs with both total and partial rows: 0


In [1058]:
total_rows_after = cleaned_stats[cleaned_stats['Team'].isin(total_team_codes)]
print(f"Number of total rows with 'Team' still showing total codes (should be 0): {len(total_rows_after)}")

Number of total rows with 'Team' still showing total codes (should be 0): 5


Ahh... We still have some entries wrong!

Check those players names:

In [1061]:
print(cleaned_stats[cleaned_stats['Team'].isin(total_team_codes)][['Player', 'Year', 'Team']])

             Player  Year Team
285   Carlos Arroyo  2005  TOT
341   Chucky Atkins  2006  TOT
3      Isaac Austin  1998  TOT
1020       Omer Ask  2018  TOT
218       Vin Baker  2004  TOT


Make manual corrections for these entries

In [1063]:
corrections = {
    ('Carlos Arroyo', 2005): 'DET',
    ('Chucky Atkins', 2006): 'MEM',
    ('Isaac Austin', 1998): 'LAC',
    ('Omer Ask', 2018): 'CHI',
    ('Vin Baker', 2004): 'NYK'
}

for (player, year), team in corrections.items():
    mask = (cleaned_stats['Player'] == player) & (cleaned_stats['Year'] == year)
    cleaned_stats.loc[mask, 'Team'] = team

Cleaned finally, sort it

In [1065]:
cleaned_stats = cleaned_stats.sort_values(by=['Year', 'Player']).reset_index(drop=True)

Also remove 'Rk' column, which is not necessary.

In [1067]:
cleaned_stats.drop(columns=['Rk'], inplace=True)

Lastly, just fill NaNs to 0.

In [1069]:
print(cleaned_stats.isna().sum())

Player       0
Pos          0
Age          2
Team         0
G            0
GS           0
MP           0
FG           0
FGA          0
FG%         51
3P           0
3PA          0
3P%       1653
2P           0
2PA          0
2P%         96
eFG%        51
FT           0
FTA          0
FT%        480
ORB          0
DRB          0
TRB          0
AST          0
STL          0
BLK          0
TOV          0
PF           0
PTS          2
Year         0
dtype: int64


In [1070]:
df.fillna(0, inplace=True)

#### Checking the Dataset

In [1072]:
print(cleaned_stats['Year'].value_counts().sort_index())

Year
1998    434
1999    433
2000    432
2001    432
2002    432
2003    419
2004    438
2005    461
2006    456
2007    449
2008    443
2009    439
2010    435
2011    450
2012    471
2013    463
2014    475
2015    485
2016    469
2017    478
2018    535
2019    524
2020    522
2021    533
2022    600
2023    539
2024    572
2025    569
Name: count, dtype: int64


In [1073]:
print(len(cleaned_stats))

13388


In [1074]:
print(f"Unique players: {cleaned_stats['Player'].nunique()}")

Unique players: 2688


There were 13388 entries but number of unique players were just 2688.

#### Finally, Save

In [1077]:
cleaned_stats.to_csv("NBA Player Stats_1998-2025.csv", index=False)
print("Saved merged dataset.")

Saved merged dataset.


## Merging Stats and Salary datasets

In [1079]:
print("Salary Columns:", merged_salary.columns.tolist())
print("Stats Columns:", cleaned_stats.columns.tolist())

Salary Columns: ['Player', 'Salary', 'Year']
Stats Columns: ['Player', 'Pos', 'Age', 'Team', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Year']


### PRE-2010

Filter salary and stats datasets to only include years 1998 to 2009

In [1082]:
salary_filtered_pre2010 = merged_salary[(merged_salary['Year'] >= 1998) & (merged_salary['Year'] < 2010)]
stats_filtered_pre2010 = cleaned_stats[(cleaned_stats['Year'] >= 1998) & (cleaned_stats['Year'] < 2010)]

Create sets of (Player, Year) tuples

In [1084]:
salary_pairs_pre2010 = set(zip(salary_filtered_pre2010['Player'], salary_filtered_pre2010['Year']))
stats_pairs_pre2010 = set(zip(stats_filtered_pre2010['Player'], stats_filtered_pre2010['Year']))

Find mismatches

In [1086]:
salary_not_in_stats_pre2010 = salary_pairs_pre2010 - stats_pairs_pre2010
stats_not_in_salary_pre2010 = stats_pairs_pre2010 - salary_pairs_pre2010

In [1087]:
print("From 1998–2009:")
print(f"(Player, Year) in salary but not in stats: {len(salary_not_in_stats_pre2010)}")
print(f"(Player, Year) in stats but not in salary: {len(stats_not_in_salary_pre2010)}")

From 1998–2009:
(Player, Year) in salary but not in stats: 235
(Player, Year) in stats but not in salary: 2091


In [1088]:
print("Number of data entries from 1998 to 2009 in stats:", len(cleaned_stats_pre2010))
print("Number of data entries from 1998 to 2009 in salary:", len(cleaned_salaries_pre2010))

Number of data entries from 1998 to 2009 in stats: 5268
Number of data entries from 1998 to 2009 in salary: 3408


### Post-2010 (Dataset that I mainly used on analysis)

Filter salary and stats datasets to only include years 2010 to 2025

In [1091]:
salary_filtered = merged_salary[(merged_salary['Year'] >= 2010)]
stats_filtered = cleaned_stats[(cleaned_stats['Year'] >= 2010)]

Create sets of (Player, Year) tuples

In [1093]:
salary_pairs = set(zip(salary_filtered['Player'], salary_filtered['Year']))
stats_pairs = set(zip(stats_filtered['Player'], stats_filtered['Year']))

Find mismatches

In [1095]:
salary_not_in_stats = salary_pairs - stats_pairs
stats_not_in_salary = stats_pairs - salary_pairs

In [1096]:
print("From 2010–2025:")
print(f"(Player, Year) in salary but not in stats: {len(salary_not_in_stats)}")
print(f"(Player, Year) in stats but not in salary: {len(stats_not_in_salary)}")

From 2010–2025:
(Player, Year) in salary but not in stats: 1682
(Player, Year) in stats but not in salary: 821


In [1097]:
cleaned_salaries_after_2010 = merged_salary[merged_salary['Year'] >= 2010]
cleaned_stats_after_2010 = cleaned_stats[cleaned_stats['Year'] >= 2010]

In [1098]:
print("Number of data entries from 2010 onwards:", len(cleaned_stats_after_2010))
print("Number of data entries from 2010 onwards:", len(cleaned_salaries_after_2010))

Number of data entries from 2010 onwards: 8120
Number of data entries from 2010 onwards: 8978


Merge

In [1100]:
merged_salaries_filtered = merged_salary[merged_salary['Year'] >= 2010]
cleaned_stats_filtered = cleaned_stats[cleaned_stats['Year'] >= 2010]

final_merged_filtered = pd.merge(merged_salaries_filtered, cleaned_stats_filtered, on=['Player', 'Year'], how='inner')
print("Filtered merged dataset shape:", final_merged_filtered.shape)

Filtered merged dataset shape: (7298, 31)


In [1101]:
final_merged_filtered.to_csv("NBA Player Stats and Salaries_2010-2025.csv", index=False)

### Additional: FULL 2000-2025

In [1103]:
final_merged = pd.merge(merged_salary, cleaned_stats, on=['Player', 'Year'], how='inner')

final_merged.to_csv("NBA Player Stats and Salaries_2000-2025.csv", index=False)